In [ ]:
#pip install small-text[transformers]

In [ ]:
#pip install setfit

In [ ]:
#pip install hnswlib

In [ ]:
#pip install openpyxl

In [ ]:
#!/bin/bash

In [ ]:
# Import the rcParams module from the matplotlib library to customize plot appearance
from matplotlib import rcParams
# Update the font size of the x-axis and y-axis labels, as well as the overall axis labels
rcParams.update({'xtick.labelsize': 14, 'ytick.labelsize': 14, 'axes.labelsize': 16})

# Import the torch and numpy libraries for numerical computations
import torch
import numpy as np

# Set a seed value for reproducibility in random number generation
seed = 2022
torch.manual_seed(seed)  # Set the seed for PyTorch
np.random.seed(seed)     # Set the seed for NumPy

# Import the logging, glob, os, and pandas libraries for file handling and data manipulation
import logging
import glob
import os
import pandas as pd

# Read the initial training data from a CSV file into a pandas DataFrame
#initial_data = pd.read_csv("initial_train_updated.CSV", delimiter='\t', index_col=False)
initial_data = pd.read_csv("initial_train_smalltext.csv", delimiter='\t', index_col=False)
print(initial_data)  # Print the initial training data

# Get the number of unique classes in the 'Label.Marius' column of the initial training data
num_classes = initial_data['label'].nunique()

print("----------------------------")

# Define the path to the 'Training Data' directory
path = r'./Training Data'

# Use the glob library to get a list of all CSV files in the specified directory and its subdirectories
all_files = glob.glob(os.path.join(path, "**/*.csv"), recursive=True)

# Read and concatenate all the CSV files into a single pandas DataFrame
new_data = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
print(new_data)  # Print the concatenated data

print("----------------------------")

# Print the first 2 training samples from the initial training data
print('First 2 training samples:\n')
for i in range(2):
    print(initial_data.iloc[i, 0], ' ', initial_data.iloc[i, 1])

# Print the first 2 future samples from the concatenated data
print('First 2 future samples:\n')
for i in range(2):
    print(new_data.iloc[i, 0], ' ', initial_data.iloc[i, 1])

print("----------------------------")

# Print the profile attribute of the new_data DataFrame 
print(new_data.example)

In [ ]:
# Import the numpy library for numerical computations
import numpy as np
# Import the LABEL_IGNORED and LABEL_UNLABELED constants from the small_text.base module
from small_text.base import LABEL_IGNORED, LABEL_UNLABELED

# Convert the 'label' column of the initial_data DataFrame into a categorical variable
initial_data['label'] = pd.Categorical(initial_data.label)
# Create a new column 'code' in the initial_data DataFrame that contains the numerical codes of the categorical labels
initial_data['code'] = initial_data['label'].cat.codes

# Print the initial_data DataFrame with the new 'code' column
print(initial_data)
print("--------")

# Create a new DataFrame 'df' by converting lists of tuples into a pandas DataFrame
# The DataFrame has three columns: 'example', 'label', and 'code'
# The 'example' column contains the values from the 'profile' column of the new_data DataFrame
# The 'label' column contains the string 'LABEL_UNLABELED' for all rows
# The 'code' column contains the numerical value of the LABEL_UNLABELED constant for all rows
df = pd.DataFrame(
    list(zip(new_data.example,  # use column "profile"
             ['LABEL_UNLABELED'] * new_data.shape[0],
             [LABEL_UNLABELED] * new_data.shape[0])),
    columns=['example', 'label', 'code'])

# Print the newly created DataFrame 'df'
print(df)
print("--------")

# Concatenate the initial_data and df DataFrames along the row axis
# The resulting DataFrame 'input_data' has a multi-level index with two levels: 'Initial Training' and 'Twitter Data'
input_data = pd.concat([initial_data, df], keys=['Initial Training', 'Twitter Data'])

# Print the concatenated DataFrame 'input_data'
print(input_data)

In [ ]:
#DELETE NA FROM  TEXTS
input_data.loc[input_data.example.isna(), 'example'] = ""

In [1]:
# Import the TextDataset class from the small_text module.
from small_text import TextDataset

# Creates an array of numbers from 0 to num_classes-1. The variable num_classes is presumably defined elsewhere in the code.
target_labels = np.arange(num_classes)

# Prints the array of target labels to the console.
print(target_labels)

# Creating an instance of the TextDataset using the `from_arrays` class method. This requires several arguments:
# 1. A list of examples obtained from the 'example' column of the input_data dataframe.
# 2. An array of codes obtained from the 'code' column of the input_data dataframe.
# 3. The target labels, which is an array of integers from 0 to num_classes-1.
train = TextDataset.from_arrays(
    input_data.example.tolist(),
    input_data.code.to_numpy(),
    target_labels=target_labels)

NameError: name 'np' is not defined

In [ ]:
# check max length
max_length = 0
for example in input_data.example:
    length = len(example)
    if length > max_length:
        max_length = length
        
print(max_length)

In [ ]:
# Import the SetFitModelArguments class from the small_text.integrations.transformers.classifiers.setfit module.
# used to define or set up arguments required for a SetFit model.
from small_text.integrations.transformers.classifiers.setfit import SetFitModelArguments

# Import the SetFitClassificationFactory class from the small_text.integrations.transformers.classifiers.factories module.
# provides a mechanism to create or instantiate SetFit classification models based on the provided arguments.
from small_text.integrations.transformers.classifiers.factories import SetFitClassificationFactory

# Define a string variable that holds the name of a pretrained sentence transformer model.
# This particular model, 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', is designed for paraphrasing tasks and supports multiple languages.
sentence_transformer_model_name = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'

# Create an instance of SetFitModelArguments using the previously defined sentence transformer model name.
# encapsulate necessary configuration or settings needed to initialize or train a SetFit model using the specified transformer model.
setfit_model_args = SetFitModelArguments(sentence_transformer_model_name)

# Create an instance of the SetFitClassificationFactory.
# This factory requires two main arguments:
# 1. The SetFit model arguments, which were defined in the previous step.
# 2. The number of classes (num_classes), which is presumably defined elsewhere in your code.
# Additionally, an optional classification_kwargs dictionary is provided, which sets a "max_seq_len" (maximum sequence length) of 128 for the classification task.
clf_factory = SetFitClassificationFactory(setfit_model_args, 
                                          num_classes, 
                                          classification_kwargs=dict({"max_seq_len":128}))  # set maximum length

In [ ]:
np.where(train.y > -1)[0] #show which profile descriptions have already been labeled

In [ ]:
from class_balancer import ClassBalancer #file in workingspace

In [ ]:
# Import various classes and functions from the small_text module.
from small_text import (
    PoolBasedActiveLearner,           # A class for pool-based active learning.
    PredictionEntropy,                # A measure or strategy based on the prediction's entropy.
    TransformerBasedClassificationFactory,  # Factory to create transformer-based classifiers.
    TransformerModelArguments,        # Arguments required for transformer models.
    random_initialization_balanced,   # A function/method for balanced initialization of data.
    BreakingTies,                     # A query strategy based on breaking ties in predictions.
    SEALS                             # Another query strategy.
)

# imports related to other query strategies and exceptions:
# from small_text.query_strategies import PoolExhaustedException, EmptyPoolException
# from small_text.query_strategies import RandomSampling, LeastConfidence, PredictionEntropy

# Define a function to initialize an active learner.
# This function simulates an initial labeling to kick-start the active learning process.
def initialize_active_learner(active_learner, y_train):
    
    # Find indices where y_train has values greater than -1.
    # labeled instances.
    indices_initial = np.where(y_train > -1)[0]
    
    # Provide the initial labeled data to the active learner.
    active_learner.initialize_data(indices_initial, y_train[indices_initial])
    
    # Print the labels of the initial labeled data.
    print(y_train[indices_initial])
    
    return indices_initial

# Set the active learning query strategy.
query_strategy = SEALS(BreakingTies())  # or also just "BreakingTies()"
# other strategies that might be used:
# query_strategy = ClassBalancer(PredictionEntropy(), ignored_classes=[10], subsample_size=None)
# query_strategy = PredictionEntropy()

# Arguments for training the SetFit model.
# Here, it's set to show a progress bar during training.
setfit_train_kwargs = {'show_progress_bar': True}

# Create an instance of the PoolBasedActiveLearner.
# It requires:
# - a factory to create classifiers (clf_factory, presumably defined in previous snippets)
# - a query strategy for active learning
# - the training data
# - training arguments specific to the SetFit model
active_learner = PoolBasedActiveLearner(
    clf_factory, 
    query_strategy, 
    train, 
    fit_kwargs={'setfit_train_kwargs': setfit_train_kwargs}
)

# Initialize the active learner with the training labels.
# Store the indices of the labeled data points.
indices_labeled = initialize_active_learner(active_learner, train.y)

In [ ]:
# Import necessary libraries and tools.
# `accuracy_score` and `f1_score` are metrics used to evaluate the performance of a machine learning model.
# `clear_output` is a utility function from IPython to clear the cell's output in Jupyter Notebooks.
from sklearn.metrics import accuracy_score, f1_score
from IPython.display import clear_output

# Set the number of queries for the active learning process. Currently, it is set to 1.
num_queries = 1

# Define a function called 'evaluate' to assess the performance of the active learner.
# The function will evaluate the learner on training data using the F1 score.
def evaluate(active_learner, train_set): 

    # Use the classifier within the active_learner to predict the labels of the train_set.
    y_pred = active_learner.classifier.predict(train_set)
    
    # This line is commented out, but if uncommented, it would predict labels for a test set.
    # y_pred_test = active_learner.classifier.predict(test)
    
    # Print the predicted labels.
    print(y_pred)
    
    # Print the true labels of the training data.
    print(train_set.y)

    # Compute and print the F1 score for the predicted labels versus the true labels.
    # The F1 score is computed using macro averaging, which calculates metrics for each label and finds their unweighted mean.
    print('Train f1: {:.2f}'.format(
        f1_score(y_pred, train_set.y, average='macro')))
    print('---')

# Create an empty list to store results.
results = []

# These lines are commented out, but they would evaluate the active learner and append the results.
# There seems to be an inconsistency in the arguments passed to the 'evaluate' function in the commented lines.
# results.append(evaluate(active_learner, train[indices_labeled], test))
# results.append(evaluate(active_learner, train[indices_labeled]))

In [ ]:
# first prediction

import csv

# Query the active learner to get indices of the top 70 samples which it wants to label.
indices_queried = active_learner.query(num_samples=70)

# Use the classifier within the active_learner to predict the class probabilities for the queried samples.
probas = active_learner.classifier.predict_proba(train[indices_queried])

# Convert the predicted class probabilities to class labels by selecting the class with the highest probability.
y_predicted = np.argmax(probas, axis=1)

# Initialize an empty list called y
y = []

# Print the predicted labels.
print(y_predicted)

# Open a new CSV file to write the predicted labels.
with open('Predicted Labels/predicted_labels_cer_1.csv', mode='w', newline='') as file:
    # Define the column names or fieldnames for the CSV.
    fieldnames = ['text', 'prediction', 'true label']
    
    # Use the csv.DictWriter to write dictionaries into the CSV file.
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header row using the defined fieldnames.
    writer.writeheader()
    
    # Iterate over the queried indices and write each sample's text, predicted label, and a placeholder for the true label.
    for j in range(len(indices_queried)):
        writer.writerow({'text': train.x[indices_queried[j]], 'prediction': y_predicted[j], 'true label': ""})

# Print the y list, which remains empty at this point.
print(y)

# Pause the program execution and prompt the user to press Enter to continue.
input("Press Enter to continue...")

# The following commented line simulates user interaction.
# In a real-world scenario, after writing the predicted labels to the CSV, an expert might manually label them.
# After this, the true labels would replace the placeholder values in the CSV.
# y = train.y[indices_queried]


In [ ]:
# 2: pickle the model
import pickle

with open('active_learner_cer.pkl', 'wb') as file:
    pickle.dump(active_learner, file)

In [ ]:
######## test
import pickle 

with open('active_learner_cer.pkl', 'rb') as file:
    active_learner = pickle.load(file)
    
print(active_learner.classifier)